# AI4Kitchen Quickstart Demo

This notebook walks through a minimal example of loading the AI4Kitchen recipe dataset and previewing a few image/text pairs. It is designed to run in Google Colab or any environment where the repository is available.

## 1. Environment setup

If you're running this notebook in Colab, uncomment the cell below to install the project dependencies directly from PyPI (or install them however you prefer). Adjust the command if you have a local wheel or specific forks.

In [ ]:
# Uncomment if running in a clean environment like Google Colab
# !pip install -q datasets pillow transformers lightning

from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
    print(f"Added {SRC_DIR} to PYTHONPATH")
else:
    print("SRC directory already on PYTHONPATH or missing.")

## 2. Load a small split of the recipe dataset

We reuse the repository's `IngredientRecipeDataset` wrapper to pull a tiny slice of the Hugging Face dataset. Feel free to change the slice or the dataset name to explore more samples.

In [ ]:
from ai4kitchen.data.dataset import IngredientRecipeDataset

dataset = IngredientRecipeDataset(split="train[:3]")
len(dataset)

## 3. Display three image/text pairs

The helper below tries multiple strategies to render an image for each example. If the dataset doesn't ship with the image bits or you don't have them locally, it will fall back to a message explaining the missing asset.

In [ ]:
from typing import Any
from IPython.display import display, Markdown
from PIL import Image

def display_example(example: Any, raw_example: dict[str, Any], index: int) -> None:
    title = raw_example.get("Title", f"Example {index + 1}")
    ingredients = example.ingredients or []
    recipe_text = example.recipe_text or "No recipe text available."

    display(
        Markdown(
            f"### {title}\n"
            f"**Ingredients:** {', '.join(ingredients) if ingredients else 'N/A'}\n\n"
            f"**Recipe:** {recipe_text[:400]}{'…' if len(recipe_text) > 400 else ''}"
        )
    )

    image_displayed = False

    raw_image = raw_example.get("image")
    if isinstance(raw_image, Image.Image):
        display(raw_image)
        image_displayed = True

    if not image_displayed:
        candidate_path = raw_example.get("Images") or raw_example.get("Image") or raw_example.get("image_path")
        if isinstance(candidate_path, str):
            path = Path(candidate_path)
            if path.exists():
                display(Image.open(path))
                image_displayed = True

    if not image_displayed and example.image_path:
        fallback_path = Path(example.image_path)
        if fallback_path.exists():
            display(Image.open(fallback_path))
            image_displayed = True

    if not image_displayed:
        display(Markdown("_Image preview unavailable in this environment. Check that the dataset includes image files and they have been downloaded._"))

for idx in range(len(dataset)):
    example = dataset[idx]
    raw_example = dataset._dataset[idx]  # type: ignore[attr-defined]
    display_example(example, raw_example, idx)
